# La Red Neuronal

In [ ]:
!pip3 install torch torchvision

In [ ]:
import numpy as np
import cv2
from keras.datasets import mnist
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

from random import randint

# Lectura y Prerocesamiento de Datos

In [ ]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# x_train = x_train.astype(float) / 255.
# x_test = x_test.astype(float) / 255.

# # extraemos los últimos 10K datos para validación
# x_train, x_val = x_train[:-10000], x_train[-10000:]
# y_train, y_val = y_train[:-10000], y_train[-10000:]

# x_train = x_train.reshape([x_train.shape[0], -1])
# x_val = x_val.reshape([x_val.shape[0], -1])
# x_test = x_test.reshape([x_test.shape[0], -1])

In [ ]:
# ####ONE HOT ENCODE####
# new_y_train = np.zeros((y_train.shape[0], 10))
# new_y_train[range(len(y_train)), y_train] = 1

# new_y_val = np.zeros((y_val.shape[0], 10))
# new_y_val[range(len(y_val)), y_val] = 1

# new_y_test = np.zeros((y_test.shape[0], 10))
# new_y_test[range(len(y_test)), y_test] = 1

In [ ]:
# np.save('x_train', x_train)
# np.save('y_train', new_y_train)
# np.save('x_val', x_val)
# np.save('y_val', new_y_val)
# np.save('x_test', x_test)
# np.save('y_test', new_y_test)

In [ ]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_val = np.load('x_val.npy')
y_val = np.load('y_val.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

# Visualizamos los datos

In [ ]:
index = randint(0, x_train.shape[0])

plt.imshow(x_train[index].reshape((28,28)), cmap='gray')
plt.show()

In [ ]:
print(y_train[index,:])

# Pytorch

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Definimos el modelo

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, hidden_layer_sizes):
        super(NeuralNetwork, self).__init__()
        
        self.hidden_layers = nn.ModuleList()
        
        for i in range(len(hidden_layer_sizes)-1):
            prev_layer_size = hidden_layer_sizes[i]
            next_layer_size = hidden_layer_sizes[i+1]
            
            self.hidden_layers.append(nn.Linear(prev_layer_size, next_layer_size))
            self.hidden_layers.append(nn.ELU())
    
    def forward(self, x):
        out = x
        for layer in self.hidden_layers:
            out = layer(out)
        return out

## Definimos la red y exploramos las dimensiones de los parámetros

In [ ]:
m, n = x_train.shape
layer_sizes = (784, 512, 10)

model = NeuralNetwork(layer_sizes)

In [ ]:
for p in model.parameters():
    print(p.shape)

# Definimos la funcion de costo y el optimizador

In [ ]:
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
num_epochs = 100
images_train = Variable(torch.from_numpy(x_train.astype(np.float32)).view(-1, 28*28))
labels_train = Variable(torch.from_numpy(np.argmax(y_train.astype(np.int64), axis=-1)))

images_val = Variable(torch.from_numpy(x_val.astype(np.float32)).view(-1, 28*28))
labels_val = Variable(torch.from_numpy(np.argmax(y_val.astype(np.int64), axis=-1)))

## Train model

In [ ]:
def print_metrics(epoch, loss, train_out, train_label, val_out, val_label):
    print('Iteración:', epoch+1, '|',
          'Costo:', loss.item(), '|', 
          'Train Acc:', accuracy_score(np.argmax(train_out.detach().numpy(), axis=1), train_label), '|', 
          'Val Acc:', accuracy_score(np.argmax(val_out.detach().numpy(), axis=1), val_label))

In [ ]:
for epoch in range(num_epochs):
    images_train.cuda()
    labels_train.cuda()
    
    optimizer.zero_grad()
    outputs = model(images_train)
    loss = criterion(outputs, labels_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        val_out = model(images_val)
        print_metrics(epoch, loss, outputs, labels_train, val_out, labels_val)

# Test Set Pytorch

In [ ]:
images_test = Variable(torch.from_numpy(x_test.astype(np.float32)).view(-1, 28*28))
labels_test = Variable(torch.from_numpy(np.argmax(y_test.astype(np.int64), axis=-1)))

In [ ]:
pred = model(images_test)
pred = np.argmax(pred.detach().numpy(), axis=1)

print(accuracy_score(pred, labels_test))
print(confusion_matrix(pred, labels_test))